# Weekly Report for Hospital Data

In [18]:
# Imports
import warnings
import pandas as pd
import psycopg
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import numpy as np

from credentials import DB_PASSWORD, DB_USER

In [19]:
# Parameters

week = "2022-9-30"

In [ ]:
# Omit warnings
warnings.filterwarnings('ignore')

In [20]:
# Connect to DB
conn = psycopg.connect(
    host="sculptor.stat.cmu.edu", dbname=DB_USER,
    user=DB_USER, password=DB_PASSWORD
)
cur = conn.cursor()

In [21]:
# Summary I & II
with conn.transaction():
    cur.execute(
        "SELECT report_date AS week, COUNT(*) AS records_loaded, "
        "SUM(total_adult_hospital_beds), SUM(total_pediatric_hospital_beds), "
        "SUM(total_adult_hospital_beds_occupied), "
        "SUM(total_pediatric_hospital_beds_occupied), "
        "SUM(inpatient_beds_occupied_covid) "
        "FROM facility_reports "
        "GROUP BY report_date "
        "ORDER BY week DESC;"
    )

records = pd.DataFrame(
    cur.fetchall(), columns=["Week", "Records Loaded", "Total Adult Hospital Beds Available", 
                             "Total Pediatric Hospital Beds Available", 
                             "Total Adult Hospital Beds In-Use", "Total Pediatric Hospital Beds In-Use",
                             "Total Hospital Beds Used by COVID cases"]
)

This is a weekly report about the hospital beds in the USA. We will use tables and plots to analyze the data.

## 1.1 Summary of Loaded Records


In this table, we list the recent five weeks hospital data upload date and rows loaded. On Oct 21, 2022, the total number of rows loaded is 4985. We can see for the past four weeks, the total number of records loaded are all around 5000.


In [22]:
# I table
import plotly.figure_factory as ff

colorscale = [[0, '#6a60a9'],[.5, '#dedcee'],[1, '#F8FAFF']]
df_1 = records[["Week", "Records Loaded"]]

fig_1 = ff.create_table(df_1, colorscale=colorscale)

fig_1.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Summary of loaded records",
        "y":0.95,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=True
)

fig_1.show()

<br />

## 1.2 Summary of Hospital Beds

In this table, we list the recent five weeks hospital beds situation. It includes the upload date, load rows number, how many adult/pediatric beds are available or in-use, total hospital beds used by Covid cases. There are around two thousand more adult beds available and in-use compared with the last week, hospitals may have 4000 new beds since last week.


In [23]:
# II table
df_2 = records
df_2.columns = ["Week", "Records Loaded", "Total Adult Hospital<br />Beds Available", 
                             "Total Pediatric Hospital<br />Beds Available", 
                             "Total Adult Hospital<br />Beds In-Use", "Total Pediatric Hospital<br />Beds In-Use",
                             "Total Hospital Beds<br />Used by COVID cases"]
                             
fig_2 = ff.create_table(df_2, colorscale=colorscale)

fig_2.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Summary of hospital beds",
        "y":0.97,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=True
)

fig_2.show()

<br />

## 1.3 Fraction of Beds Currently in use by Hospital Quality Rating

This table includes hospital’s rating, percentage of beds in use compared with the total number of beds of that hospital. For those hospitals have highest rating, 66% adult and pediatric beds are in-use, 67% ICU beds are in-use.


In [24]:
# III
with conn.transaction():
    cur.execute(
        "SELECT rating, "
        "ROUND(SUM(total_adult_hospital_beds_occupied)/SUM(total_adult_hospital_beds)*100, 2), "
        "ROUND(SUM(total_pediatric_hospital_beds_occupied)/SUM(total_pediatric_hospital_beds)*100, 2), "
        "ROUND(SUM(total_icu_beds_occupied)/SUM(total_icu_beds)*100, 2) "
        "FROM facility_information info "
        "JOIN quality_ratings rate "
        "ON info.facility_id = rate.facility_id "
        "JOIN facility_reports "
        "ON info.facility_id = hospital_pk "
        "WHERE rating IS NOT NULL "
        "AND total_adult_hospital_beds_occupied < total_adult_hospital_beds "
        "AND report_date = %(week)s"
        "GROUP BY rating "
        "ORDER BY rating DESC;",
        {"week": pd.to_datetime("2022-10-21")}
    )

ratings = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Rating", "Percent % Adult Hospital Beds In-Use", 
                             "Percent % Pediatric Hospital Beds In-Use",
                             "Percent % ICU Beds In-Use"]
)

In [25]:
# III - table

df_2_2 = ratings

fig_2_2 = ff.create_table(df_2_2, colorscale=colorscale)

fig_2_2.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Fraction of beds currently in use by hospital quality rating",
        "y":0.96,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=True
)

fig_2_2.show()

<br />

## 1.4 Number of Beds in use of all cases and covid cases

This bar plot shows the total number of hospital beds used per week for all causes and covid cases in the recent five weeks. On Oct 21, 2022, 2626 more beds were in use for all cases, and there are 421 beds occupied by patients have Covid.


In [26]:
# IV
with conn.transaction():
    # Extract the number of beds in use    
    cur.execute("SELECT report_date, SUM(total_occupied_0) "
                "AS total_occupied_all, "
                "SUM(inpatient_beds_occupied_covid) "
                "AS total_occupied_covid FROM "
                "(SELECT report_date, inpatient_beds_occupied_covid, "
                "total_adult_hospital_beds_occupied + "
                "total_pediatric_hospital_beds_occupied + "
                "total_icu_beds_occupied + "
                "inpatient_beds_occupied_covid AS total_occupied_0 "
                "FROM facility_reports) AS a "
                "GROUP BY report_date "
                "ORDER BY report_date;")
    data_used = pd.DataFrame(
        cur.fetchall(), columns=["report_date", 
                                 "all_used_beds",
                                 "covid_used_beds"]
        )


# Make the bar plot
fig_4_bar = go.Figure()
fig_4_bar.add_trace(go.Bar(
    x=data_used['report_date'],
    y=data_used['all_used_beds'],
    text=data_used['all_used_beds'],
    textposition='auto',
    name='All cases',
    marker_color='#41D3BD'
))
fig_4_bar.add_trace(go.Bar(
    x=data_used['report_date'],
    y=data_used['covid_used_beds'],
    text=data_used['covid_used_beds'],
    textposition='auto',
    name='COVID cases',
    marker_color='#379392'
))

fig_4_bar.update_layout(
    title={
        "text":"Number of beds in use of all cases and covid cases",
        "y":0.88,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    plot_bgcolor='#D8E6E7')

fig_4_bar.show()

These pie charts show the percentage of hospital beds used per week for all causes and covid cases. We can see in the recent five weeks, the percentage didn’t change too much, all cases have around 95% of beds in-use and 5% beds in-use for Covid cases.

In [27]:
# Make the pie chart
labels = ['All cases', 'COVID cases']

# Define color sets of paintings
colors1 = ['#88dba3', '#cff0da']
colors2 = ['#4ea1d3', '#d8e9ef']
colors3 = ['#6a60a9', '#dedcee']
colors4 = ['#fbd14b', '#fffcf0']
colors5 = ['#ee6e9f', '#fec9c9']

# Create subplots, using 'domain' type for pie charts
specs = [[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}], 
        [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]
fig_4_pie = make_subplots(rows=2, cols=3, specs=specs)

# Define pie charts
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][0],
                    data_used["covid_used_beds"][0]], name=str(data_used['report_date'][0]),
                     marker_colors=colors1), 1, 1)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][1],
                    data_used["covid_used_beds"][1]], name=str(data_used['report_date'][1]),
                     marker_colors=colors2), 1, 2)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][2],
                    data_used["covid_used_beds"][2]], name=str(data_used['report_date'][2]),
                     marker_colors=colors3), 1, 3)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][3],
                    data_used["covid_used_beds"][3]], name=str(data_used['report_date'][3]),
                     marker_colors=colors4), 2, 1)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][4],
                    data_used["covid_used_beds"][4]], name=str(data_used['report_date'][4]),
                     marker_colors=colors5), 2, 2)

# Tune layout and hover info
fig_4_pie.update_traces(hole=.4, hoverinfo='label+percent+name', textinfo='none')
fig_4_pie.update_layout(
    title={
        "text":"Number of beds in use of all cases and covid cases",
        "y":0.88,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    showlegend=False,
    annotations=[dict(text='9-23', x=0.132, y=0.825, font_size=13, showarrow=False),
                 dict(text='9-30', x=0.866, y=0.825, font_size=13, showarrow=False),
                 dict(text='10-07', x=0.5, y=0.825, font_size=13, showarrow=False),
                 dict(text='10-14', x=0.5, y=0.18, font_size=13, showarrow=False),
                 dict(text='10-21', x=0.131, y=0.18, font_size=13, showarrow=False)]
    )

fig_4_pie = go.Figure(fig_4_pie)
fig_4_pie.show()

<br />

## 2.1 The number of COVID cases by state

This map shows how many Covid cases we have in the USA until Oct 21, 2022. Darker the color, this state has more people have Covid. In New York state, there are 1841 people have Covid.


In [28]:
# Additional I - COVID map
with conn.transaction():
    # Get the data for COVID map in each state
    cur.execute("SELECT state, report_date, SUM(total_covid_0) "
                "AS total_covid FROM "
                "(SELECT hospital_pk, report_date, "
                "adult_icu_patients_confirmed_covid "
                "+ inpatient_beds_occupied_covid AS total_covid_0 "
                "FROM facility_reports "
                "GROUP BY report_date, hospital_pk) AS a "
                "INNER JOIN "
                "(SELECT facility_id, state "
                "FROM facility_information) AS b "
                "ON a.hospital_pk = b.facility_id "
                "GROUP BY state, report_date "
                "ORDER BY state;")
    
    data_map2 = pd.DataFrame(cur.fetchall(), columns=["state", "week", "beds"])


data_map_tw = data_map2.iloc[np.where(
    data_map2["week"]==pd.Timestamp(week))[0],:]

# Make the map
fig_map2 = go.Figure(data=go.Choropleth( 
    locations=data_map_tw['state'], 
    z = data_map_tw['beds'].astype(float), # Set fill color
    locationmode = 'USA-states', # Set the country 
    colorscale = 'Reds', # Set the legend color
    colorbar_title = "Number", # Legend title
)) 

fig_map2.update_layout(
    title={
        "text":"The number of COVID cases by state" + ": " + week,
        "y":0.95,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    geo_scope='usa', # Set country
    autosize=True,
    margin=dict(t=30,b=25,l=0,r=0)
)


/var/folders/vw/0kbg8m6n26sdbzft__pgcwc80000gn/T/ipykernel_20838/477321367.py:22: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



<br />

## 2.2 State COVID cases changes

This table listed all the states Covid cases changes status. We didn’t get data from Guam and the Virgin Islands, so it shows “None” on the top. Illinois has 133 more Covid cases compared with last week.


In [29]:
# Additional - II - State COVID increase
with conn.transaction():
    cur.execute(
        "SELECT tw.state, tw.covidcase - lw.covidcase AS coviddiff "
        "FROM ("
        # This weeks data
        "SELECT state, SUM(inpatient_beds_occupied_covid) as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(week)s "
        "GROUP BY state) tw "
        "JOIN ("
        # Last weeks data
        "SELECT state, SUM(inpatient_beds_occupied_covid) as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(lastweek)s "
        "GROUP BY state) lw "
        "ON tw.state = lw.state "
        "ORDER BY coviddiff DESC;",
        {
            "week": pd.Timestamp(week),
            "lastweek": pd.Timestamp(week) - pd.Timedelta(days=7)
        }
    )

statecovid = pd.DataFrame(
    cur.fetchall(), columns=["States", "Change in Covid Case"]
)

In [30]:
# Additional - II table
df_a_2 = statecovid

fig_a_2 = ff.create_table(df_a_2, colorscale=colorscale)

fig_a_2.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"State COVID cases changes",
        "y":0.995,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=True
)

fig_a_2.show()

<br />

## 2.3 Hospitals with largest changes in COVID cases

In this table, we list the top five hospitals with the largest changes in Covid cases, with their information. Carolinas Medical center has the largest changes. What interesting is that it’s located in North Carolina, where doesn’t have lots of Covid cases changes.


In [31]:
# Additional - III - Top Hospital COVID increase
with conn.transaction():
    cur.execute(
        "SELECT info.facility_name, tw.covidcase - lw.covidcase AS coviddiff, "
        "address, city, state, zipcode "
        "FROM ("
        # This weeks data
        "SELECT facility_id, inpatient_beds_occupied_covid as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(week)s "
        ") tw "
        "JOIN ("
        # Last weeks data
        "SELECT facility_id, inpatient_beds_occupied_covid as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(lastweek)s "
        ") lw "
        "ON tw.facility_id = lw.facility_id "
        "JOIN facility_information info "
        "ON info.facility_id = tw.facility_id "
        "WHERE tw.covidcase - lw.covidcase IS NOT NULL "
        "ORDER BY coviddiff DESC "
        "LIMIT 5",
        {
            "week": pd.Timestamp(week),
            "lastweek": pd.Timestamp(week) - pd.Timedelta(days=7)
        }
    )

hospitalcovid = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Change in Covid Case",
                             "Address", "City", "State", "Zipcode"]
)

In [32]:
# Additional - III table
hospitalcovid_new = hospitalcovid

hospitalcovid_new["Hospital Name"] = [
    "CAROLINAS MEDICAL CENTER<br />/BEHAV HEALTH",
    "NEW YORK-PRESBYTERIAN<br />HOSPITAL",
    "GEISINGER MEDICAL CENTER",
    "HOUSTON METHODIST HOSPITAL",
    "MAINE MEDICAL CENTER"]

df_a_3 = hospitalcovid_new

fig_a_3 = ff.create_table(df_a_3, colorscale=colorscale)

fig_a_3.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Hospitals with largest changes in COVID cases",
        "y":0.97,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=True
)

fig_a_3.show()

<br />

## 2.4 Hospital with no reported data

It’s a table of hospital names which did not report data last week, with the last reported date.
There are 17 hospitals doesn’t have data this week (2022-10-21).


In [33]:
# Additional - IV - Hospital with no data
with conn.transaction():
    cur.execute(
        "SELECT facility_name, MAX(report_date) "
        "FROM facility_information "
        "JOIN facility_reports ON "
        "facility_id = hospital_pk "
        "GROUP BY facility_id "
        "HAVING MAX(report_date) < %(week)s "
        "ORDER BY MAX(report_date) DESC;",
        {"week": pd.Timestamp(week)}
    )

nodata = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Last reported Date"]
)

In [36]:
# Additional - IV table
df_a_4 = nodata

fig_a_4 = ff.create_table(df_a_4, colorscale=colorscale)

fig_a_4.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Hospital with no reported data",
        "y":0.98,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=True
)

fig_a_4.show()